In [1]:
"""Sandbox module."""

import numpy as np
from data_generation import (
    gen_discretized_distributions,
    gen_grids_and_parameters,
)

from misc import (
    riemann_sum_arrays,
    trunc_norm_pdf,
    cdf_from_density,
    quantile_from_density,
)

%matplotlib inline

In [2]:
import warnings

In [3]:
# Set up data
n = 200
gridnum = 1000
truncation_point = 3

grid_pdfs, grid_qfs, mus, sigmas = gen_grids_and_parameters(
    n, gridnum, truncation_point, delta=0,
)

In [4]:
def qd_from_dens(dens, dsup=None, qdsup=None):
    """Compute quantile densities directly from densities.

    'Inspired' from dens2qd in fdadensity package in R.

    """
    # Validate input
    eps = 1e-3
    boundaries = [np.min(qdsup), np.max(qdsup)]
    if not np.allclose(boundaries, [0, 1], atol=eps):
        msg = f"Please check the support of the QF domain's boundaries: {boundaries}"
        raise ValueError(msg)

    integral_dens = riemann_sum_arrays(dsup, array=dens, axis=-1, cumsum=True)
    deviations_from_1 = abs(integral_dens[...,-1] - 1)
    if np.any(deviations_from_1 > eps):
        warnings.warn(
            f"Not all provided densities integrate to 1 with tolerance {eps}!"
            f"\n Max case of deviation is: {deviations_from_1.max()}"
            f"\n In position: {deviations_from_1.argmax()} "
            "\n Performing normalization...",
        )
        dens /= integral_dens[...,-1][..., np.newaxis]

    if useSplines:
        dens_sp = interp1d(dSup, dens, kind='cubic')
        qdtemp = np.zeros_like(dSup)
        for i in range(1, len(dSup)):
            qdtemp[i], _ = quad(dens_sp, dSup[i-1], dSup[i])
        qdtemp = np.cumsum(qdtemp)
    else:
        qdtemp = np.cumsum(np.trapz(dens, dSup))


    qdtemp = integral_dens
    qdens_temp = 1 / dens
    ind = np.unique(qdtemp, return_index=True, axis=-1)[1]
    qdtemp = qdtemp[..., ind]
    qdens_temp = qdens_temp[..., ind]

    qd = np.zeros(dens.shape)
    for i in range(len(dens)):
        qd[i] = np.interp(qdsup, qdtemp[i], qdens_temp[i])

    integral_qd = riemann_sum_arrays(qdsup, qd, axis=-1, cumsum=False)
    qd *= np.ptp(dsup) / integral_qd[..., np.newaxis]

    return qd

In [5]:
# def gen_discretized_distributions(grid_pdfs, grid_qfs, mus, sigmas, truncation_point):
#     """Generate discretized pdfs, cdfs, qfs, and qdfs."""
#     # Truncated pdfs
#     pdfs_discretized = trunc_norm_pdf(
#         grid_pdfs[:, np.newaxis],
#         mus,
#         sigmas,
#         -truncation_point,
#         truncation_point,
#     )

#     # Truncated cdfs
#     cdfs_discretized = cdf_from_density(
#         grid_pdfs,
#         pdfs_discretized,
#         axis=-1,
#     )

#     # Truncated qfs
#     qfs_discretized = quantile_from_density(
#         pdfs_discretized,
#         grid_pdfs,
#         grid_qfs,
#     )

#     # Truncated qdfs
#     qdfs_discretized = qd_from_dens(
#         pdfs_discretized, dsup=grid_pdfs, qdsup=grid_qfs,
#     )

#     return pdfs_discretized, cdfs_discretized, qfs_discretized, qdfs_discretized


In [6]:
# Generate distributions
pdfs_discretized, cdfs_discretized, qfs_discretized, qdfs_discretized = (
    gen_discretized_distributions(grid_pdfs, grid_qfs, mus, sigmas, truncation_point)
)

c:\Coding\thesis_frechet_fda\src\frechet_fda\misc.py:71: UserWarning: Not all provided densities integrate to 1 with tolerance 1e-05!
 Max case of deviation is: 0.006087386759915159
 In position: 40
 Performing normalization...
  warnings.warn(


In [11]:
qdfs_discretized[0][499], pdfs_discretized[0][499]

(1.2826654560707763e-07, 0.8689204433808188)

In [12]:
(np.log(qdfs_discretized[0]) + np.log(np.interp(qfs_discretized[0], grid_pdfs, pdfs_discretized[0])))

array([-1.60096591e+01,  4.59597600e-01,  1.00861520e+00,  1.29115906e+00,
        1.46529294e+00,  1.58037114e+00,  1.65826471e+00,  1.71060056e+00,
        1.74435405e+00,  1.76396465e+00,  1.77248994e+00,  1.77210196e+00,
        1.76436268e+00,  1.75053761e+00,  1.73153451e+00,  1.70809262e+00,
        1.68082978e+00,  1.65023146e+00,  1.61668049e+00,  1.58054467e+00,
        1.54212312e+00,  1.50160369e+00,  1.45926342e+00,  1.41522162e+00,
        1.36969488e+00,  1.32277738e+00,  1.27460670e+00,  1.22530557e+00,
        1.17496572e+00,  1.12364909e+00,  1.07144836e+00,  1.01843942e+00,
        9.64683709e-01,  9.10237329e-01,  8.55151687e-01,  7.99474007e-01,
        7.43247783e-01,  6.86513170e-01,  6.29307327e-01,  5.71664722e-01,
        5.13617386e-01,  4.55195154e-01,  3.96417249e-01,  3.37300571e-01,
        2.77884452e-01,  2.18191843e-01,  1.58244305e-01,  9.80621313e-02,
        3.76295488e-02, -2.30065858e-02, -8.38254460e-02, -1.44810833e-01,
       -2.05987998e-01, -

In [9]:
riemann_sum_arrays(grid_pdfs, pdfs_discretized[0])

1.0

In [10]:
def log_qd_transformation(qd, lqdsup, lqdsup):
    """Log quantile density transformation"""
    eps = 1e-4
    integral_qd = riemann_sum_arrays(qdsup, array=qd, axis=-1, cumsum=False)
    if not np.isclose(integral_qd[-1], np.ptp(dsup), atol=eps):
        msg = ()"Quantile Density does not integrate to the range of the densities with "
        f"tolerance {eps}."
        f"\n Integral is: {integral_qd[...,-1]}"
        f"\n Range is: {np.ptp(dsup)}")
        raise ValueError(msg)
    return np.log(qd)


def inverse_transformation():

SyntaxError: unmatched ')' (3820928570.py, line 9)